# Multiple choice case

In [ ]:
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)
from ollama_models import ollama_models
models = ollama_models()

models

model = models[0]

from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm

question = load_saved_questions([20683])[0]

from community_forecast import community_forecast
import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
from gather_research_and_set_prompt import gather_research_and_set_prompt
from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
from tqdm import tqdm
import time

print('START model', model, 'id', id)
start_time = time.time()
questions = [question]
id = question.id_of_question
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)

df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
    'question_resolution_criteria', 'question_fine_print', 'question_type', 
     'question_description',
    'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()

df2, rag = gather_research_and_set_prompt(df1, True, model)

df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_type', 'question_options',
         'question_description','crowd',
          'research', 'asknews']].copy()

row = df3.iloc[0]

row

for item, value in row.items():
    print(item.upper())
    print()
    print(str(value)[0:200])
    print('===============================')
    print()

prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You will analyse and make a prediction about this question:

```title
{row.title}
```

You take into consideration this news:

```news
{row.asknews}

You take into consideration also this research:

```research
{row.research}
```

This is a multiple choice forecast.  There are N choices.  Each choice is a mutually exclusive event. 
You supply a forecast giving the percentage likelihood that the given event is likely to occur, to the exclusion of the other events.  
The events are:

```choices
{row.question_options}
```

We discriminate between the events as follows:

```resolution_criteria
{row.question_resolution_criteria}
```

The last thing you write is your final probabilities for the possible events in order as:

Choice_A: Probability_A
Choice_B: Probability_B
...
Choice_N: Probability_N

where you must assign your probabilities so they add up to 100: Probability_A + Probability_B + ... + Probability_N = 100.
"""

print(prompt2)

answer_mistral = call_local_llm(prompt2, model)
answer_mistral2 = call_local_llm(prompt2, model)
answer_mistral3 = call_local_llm(prompt2, model)

print(answer_mistral)

from extract_forecast import *

options = eval(row.question_options) 
options

forecasts = [generate_multiple_choice_forecast(options,extract_option_probabilities_from_response(x, options))
             for x in [answer_mistral, answer_mistral2, answer_mistral3]]
forecasts

from median_dictionaries import median_dictionaries

forecast = median_dictionaries(forecasts)

forecast

prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

rationale = call_local_llm(prompt, model)



print(rationale)

row.forecast = rationale

row.prediction = forecast

row.prediction

In [59]:
row.crowd

{'AI-Fizzle': 0.1271267605633803,
 'Futurama': 0.29274775928297053,
 'AI-Dystopia': 0.28000768245838664,
 'Singularia': 0.1729910371318822,
 'Paperclipalypse': 0.1271267605633803}

In [60]:
from error import error

In [61]:
error(row)

0.28765685019206144